# DDPM: Denoising Diffusion Probabilistic Models

- **DDPM** 논문은 **Diffusion model**의 인기 상승에 큰 기여를 하였다.

# 1. Introduction

## 1.1. Generative Model

<img src="01.png" width=70% height=70%>  

(출처: https://lilianweng.github.io/posts/2021-07-11-diffusion-models/)

- Generative model(생성 모델)에는 GAN, VAE, Flow-based, Diffusion model 등이 있다.
    - **GAN(Generative Adversarial Network)**: GAN은 생성적 적대 신경망으로, 생성자(Generator)가 임의의 latent space로부터 데이터를 생성해내면 판별자(Discriminator)가 실제 데이터인지 아닌지 판별하여 서로 경쟁하면서 데이터를 생성해내는 모델이다.
    - **VAE(Variational Autoencoder)**: VAE는 오토인코더의 형태를 갖추지만 latent space가 가우시안 분포를 갖도록 하여 더 효과적으로 데이터를 생성해낼 수 있는 생성 모델이다. ELBO(Evidence of Lower BOund)를 손실함수로 하여 이를 최대화하는 방향으로 학습한다.
    - **Flow-based models**: Flow-based models은 실제 데이터로부터 latent space가 가우시안 분포가 되도록 하는 가역함수들의 흐름을 합성하고, 그 함수들의 합성의 역함수로 데이터를 생성해내는 생성 모델이다.
    - **Diffusion models**: Diffusion 모델은 데이터에 점진적으로 노이즈를 추가하여 가우시안 분포를 갖는 latent space를 만들고, 데이터를 생성하는 과정은 이와 반대로 latent space에서 노이즈를 제거하는 방식으로 작동하는 생성 모델이다.

- GAN, VAE, Flows 모델은 상당히 높은 품질의 이미지와 오디오를 생성하는 데에 성공했다.
- 그러나 Diffusion이 높은 품질의 데이터를 생성한다는 논문이나 보고는 없었는데, 이 논문에서 좋은 성능을 보임을 확인한다.

## 1.2. Markov Chain

- **Markov Chain**: 마르코프 체인은 현재 상태가 오로지 직전의 상태에만 영향을 받는다는 마르코프 속성을 따르는 모델링을 의미한다. (1차 마르코프 체인)
$$
P(x_t|x_{t-1}, x_{t-2}, \dots, x_{0}) = P(x_t|x_{t-1})
$$

- 여러 모델링을 위해 현재 상태가 과거의 n개의 상태에만 영향을 받는다는 n-차 마르코프 체인을 고려하기도 한다.

- Markov Chain을 가정하면 다음이 성립한다.
$$
P(x_0, x_1, \dots, x_T) = P(x_0) \prod_{t=1}^{T}P(x_t|x_{t-1})
$$

<img src="02.jpg" width=70% height=70%>

## 1.3. Diffusion Model

- **Diffusion Model**: 유한한 시간 이후에 데이터와 일치하는 샘플을 생성하기 위해 Variational inference를 사용하여 훈련하는 매개변수화된 Markov Chain
    - 이를 위해 데이터에 가우시안 Noise를 추가하여 완전한 Noise로 만들고, 샘플링은 이와 반대되는 Denoising 과정을 거친다.

<img src="03.png" width=70% height=70%>

- Diffusion 모델은 쉽게 정의할 수 있고 효율적으로 훈련할 수 있다.
    - 노이즈를 추가하고 제거하는 과정이 직관적이며, 복잡한 구조를 필요로 하지 않는다.

# 2. Background

- 먼저 다음과 같이 정의한다.
1. $x_0$: 실제 데이터 ($x_0 \sim q(x_0)$)
2. $x_T$: 완전한 Noise ($x_T \sim N(x_T; 0, I)$)
3. $x_t(t=1, 2, \dots, T)$: 가우시안 noise가 추가된 latent space

## 2.1. Forward Process

<img src="03.png" width=70% height=70%>

- $q(x_t|x_{t-1})$은 이전 스텝에서 노이즈를 추가했을 때 현재 스탭의 데이터의 분포이다.

- $q(x_{1:T}|x_{0})$: **forward process** 또는 **diffusion process**라 부르며, 점진적으로 가우시안 noise를 추가하는 Markov Chain이다.
- 다음과 같이 $q(x_t|x_{t-1})$을 정의한다.
$$
q(x_{1:T}|x_{0}) := \prod_{t=1}^{T} q(x_{t}|x_{t-1}) \\
q(x_t|x_{t-1}) := N(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t I) \, (\beta_t \in (0, 1))
$$

- $\beta_t$는 Noise의 정도를 의미한다. 1에 가까울수록 Noise가 크고, 0에 가까울수록 Noise가 없다.
    - 이 논문에서는 $\beta_t$를 0.0001에서 0.02 사이의 값으로 설정한다.
    - $\beta_t$는 학습 파라미터로 사용하거나 하이퍼 파라미터로 사용할 수 있다. 이 논문에서는 하이퍼 파라미터로 사용하여 고정한다.

<img src="04.jpg" width=70% height=70%>

- $q(x_t|x_{t-1}) := N(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t I)$로 정의한 이유는 다음과 같다.

<img src="05.jpg" width=70% height=70%>

- Markov Chain이라면 $q(x_{1:T}|x_{0}) = \prod_{t=1}^{T} q(x_{t}|x_{t-1})$ 식이 성립하는 이유는 다음과 같다.

<img src="06.jpg" width=50% height=50%>

- noising 과정에서 $t$step 건너뛰어 다음과 같이 $x_t$를 바로 샘플링할 수도 있다.
$$
q(x_t|x_0) = N(x_t; \sqrt{\bar{\alpha_t}}x_0, (1-\bar{\alpha_t})I) \\
\alpha_t := 1-\beta_t \quad \bar{\alpha_t} := \prod_{s=1}^{t} \alpha_s
$$

- 위의 식이 성립하는 이유는 다음과 같다.

<img src="07.jpg" width=70% height=70%>

## 2.2. Reverse Process

<img src="03.png" width=70% height=70%>

- $p_{\theta}(x_{t-1}|x_{t})$: 현재 스텝에서 노이즈를 제거했을 때 이전 스탭의 데이터의 분포이다.
    - 우리는 $p_{\theta}(x_{t-1}|x_{t}) \sim q(x_{t-1}|x_{t})$가 되기를 희망한다.

- $p_{\theta}(x_{0:T})$: **reverse process**로 부르며, $p(x_T) = N(x_T; 0, I)$로부터 시작하고 가우시안 noise로부터 복원하는 Markov Chain이다.
- $p_{\theta}(x_{t-1}|x_t)$는 다음과 같이 정의할 수 있다.
$$
p_{\theta}(x_{0:T}) := p(x_T)\prod_{t=1}^{T} p_{\theta}(x_{t-1}|x_t) \\
p_{\theta}(x_{t-1}|x_t) := N(x_{t-1}; \mu_{\theta}(x_t, t), \Sigma_{\theta}(x_t, t))
$$

- 순방향(forward) $q(x_t|x_{t-1})$이 가우시안 분포를 따르면 역방향(backward)도 $p_{\theta}(x_{t-1}, x_{t})$가 가우시안 분포를 따른다는 것은 선행연구([53] Deep Unsupervised Learning using Nonequilibrium Thermodynamics)의 Appendix C에서 증명되었다.

- Markov Chain이라면 $p_{\theta}(x_{0:T}) = p(x_T)\prod_{t=1}^{T} p_{\theta}(x_{t-1}|x_t)$가 성립하는 것은 자명하다.

- 우리의 목표는 다음 likelihood를 최대화하는 것이다.
$$
p_{\theta}(x_0) = \int p_{\theta}(x_{0:T}) dx_{1:T}
$$

## 2.3. Negative Log Likelihood



- 이 모델의 **Negative Log Likelihood**는 다음과 같다.
$$
E_{x_T \sim q(x_T|x_0)} (-\log p_{\theta}(x_0)) = E_q \Big[ D_{KL}(q(x_{T}|x_{0})||p(x_{T}))+\sum_{t>1}D_{KL}(q(x_{t-1}|x_t, x_0)||p_{\theta}(x_{t-1}|x_{t}))-\log p_{\theta}(x_0|x_1) \Big]
$$

- 위와 같이 계산되는 이유는 다음과 같다.

**이전 버전**

<img src="08.jpg" width=100% height=100%>

<img src="09.jpg" width=100% height=100%>

<img src="09_1.jpg" width=100% height=100%>

<img src="09_2.jpg" width=100% height=100%>

<img src="09_4.jpg" width=100% height=100%>

**Understanding Diffusion Models 논문에서 소개된 방법**

<img src="13_1.jpg">

<img src="13_2.jpg">

<img src="13_3.jpg">

<img src="13_4.jpg">

- 위 유도식에서 나오는 $q(x_{t-1}|x_t, x_0)$는 다음과 같다.

<img src="10.jpg">

<img src="11.jpg">

<img src="12.jpg">

# 3. Diffusion models and denoising autoencoders

- Diffusion model은 생각보다 구현에 있어서 자유롭다.

## 3.1. Forward process and $L_T$

- 우리는 Forward process 중 $q(x_t|x_{t-1}) := N(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t I) \, (\beta_t \in (0, 1))$에서 $\beta_t$를 고정시킨다. $\beta_1=10^{-4}$에서 $\beta_T=0.02$까지 선형적으로 증가하도록 설정한다.
    - 이에 따라 $q$는 학습 가능한 파라미터가 존재하지 않는다. 즉, 상수 취급 가능하다.
- 손실함수 중 $L_T=D_{KL}(q(x_T|x_0)||p(x_T))$는 $q(x_T|x_0)$와 $p(x_T)$모두 상수이므로 이 항은 무시해도 된다.

## 3.2. Reverse process and $L_{1:T-1}$

- Reverse process 중 $p_{\theta}(x_{t-1}|x_t) = N(x_{t-1}; \mu_{\theta}(x_t, t), \Sigma_{\theta}(x_t, t)) (1<t<T)$를 선택해야 한다.
- $\Sigma_{\theta}(x_t, t) = \sigma_t^2 I$에서 $\sigma_t^2 = \beta_t$를 택하거나 $\sigma_t^2 = \widetilde{\beta}_t$를 택할 수 있다.
- 그러면 $L_{t-1}$은 다음과 같이 정리할 수 있다.

<img src="14.jpg">

<img src="15.jpg">

<img src="16.jpg">

**[참고 자료]**
1. Denoising Diffusion Probabilistic Models, 논문: https://arxiv.org/abs/2006.11239
2. Deep Unsupervised Learning using Nonequilibrium Thermodynamics, (선행)논문: https://arxiv.org/abs/1503.03585
3. Diffusion Model 수학이 포함된 tutorial, Youtube: https://www.youtube.com/watch?v=uFoGaIVHfoE
4. PR-409: Denoising Diffusion Probabilistic Models, Youtube: https://www.youtube.com/watch?v=1j0W_lu55nc
5. What are Diffusion Models?, Github: https://lilianweng.github.io/posts/2021-07-11-diffusion-models/
6. [Paper Review] DDPM: Denoising Diffusion Probabilistic Models 논문 리뷰, Github: https://happy-jihye.github.io/diffusion/diffusion-1/
7. [논문공부] Denoising Diffusion Probabilistic Models (DDPM) 설명, Blog: https://developers-shack.tistory.com/8
8. Markov Chain, pdf: https://www.stat.auckland.ac.nz/~fewster/325/notes/ch8.pdf
9. Purpose of scaling mean by $\sqrt{1-\beta_t}$ in forward diffusion process, StackExchange: https://stats.stackexchange.com/questions/600127/purpose-of-scaling-mean-by-sqrt1-beta-t-in-forward-diffusion-process
10. Understanding Diffusion Models: A Unified Perspective, Paper: https://arxiv.org/abs/2208.11970
11. Fubini's theorem, Wikipedia: https://en.wikipedia.org/wiki/Fubini's_theorem
12. Jensen's inequality, Wikipedia: https://en.wikipedia.org/wiki/Jensen%27s_inequality
13. Understanding Diffusion Models: A Unified Perspective, 논문: https://arxiv.org/abs/2208.11970